# Импорт библиотек

In [33]:
import mlflow
from mlflow.tracking import MlflowClient

import os

import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from transformers import pipeline
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

from datasets import Dataset

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import *

import joblib
import cloudpickle

import nltk
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

from nltk.corpus import stopwords
from functools import lru_cache

from pymorphy3 import MorphAnalyzer
import re
import emoji



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Smart\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Эксперименты с датасетами

## Первый эксперимент (Лемматизация, удаление знаков пунктуации)

In [7]:
#!   mlflow server --host 127.0.0.1 --port 8080

In [8]:
# Подключаемся к серверу, запущенным на локальном хосте

mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [9]:
analyzer = MorphAnalyzer(lang = 'ru')
stop_words = nltk.corpus.stopwords.words('russian')
stop_words_cleaned = []

for i in stop_words:
    if i != 'не' and i != 'нет' and i != 'никогда':
        stop_words_cleaned.append(i)


@lru_cache(maxsize=100_000)
def lemmatization(text):
    return analyzer.parse(text)[0].normal_form

def preprocess_text(text):

    text = re.sub(r'[\n\r\t]', ' ', text)
    
    text = re.sub(r'\bn(\w+)', r'\1', text)  # nчипсы -> чипсы
    
    text = re.sub(r'[^\w\s\d]', '', text)
    
    text = re.sub(r'\s+', ' ', text).strip()



    text_tokens = nltk.word_tokenize(text)
    sentence = []

    for i in text_tokens:
        if i not in stop_words_cleaned:
            sentence.append(lemmatization(i))

    return ' '.join(sentence)


print(preprocess_text('Всем привет! Какое же неприятное место, нет?\n'))



весь привет какой неприятный место нет


In [10]:
with mlflow.start_run(run_name='First dataset'):
    
    mlflow.set_tag('Dataset_version', '1.0.0')

    df_annotations_1 = pd.read_json("project-8-at-2025-11-01-11-43-483e368c.json")
    df_annotations_2 = pd.read_json("project-9-at-2025-11-01-11-51-0428f45f.json")
    df_annotations_3 = pd.read_json("project-10-at-2025-11-06-14-16-bb9bc9cd.json")

    df_annotations = pd.concat([df_annotations_1, df_annotations_2, df_annotations_3])

    df = pd.DataFrame(columns = ['span', 'label'])

    for mark in df_annotations['aspect_sentiment']:
        for param in range(len(mark)):
            span = mark[param]['text']
            label = mark[param]['labels'][0]

            df.loc[len(df)] = [span, label]

    df['span'] = df['span'].apply(preprocess_text)
    
    
    display(df)


    file = df.to_csv('First_version.csv')
    
    
    mlflow.log_artifact('First_version.csv', 'datasets')

    with open('preprocess_text_first.pkl', 'wb') as f: 
        preprocess_pickle = cloudpickle.dump(preprocess_text, f)

    mlflow.log_artifact('preprocess_text_first.pkl', 'functions')


,span,label
0,вкус реально отличный,ВКУС_POSITIVE
1,я есть чипсы очень долго год 2 назад советоват...,O
2,хотеться купить очень последний время стать оч...,O
3,вкус иметь каждый 2 пачка,O
4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...
2063,итогчипсы лейс flamin hot острый креветка васа...,ВКУС_POSITIVE
2064,у привлекательный аппетитный вид,ПАЧКА_POSITIVE
2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,к покупка рекомендовать,ВКУС_POSITIVE


🏃 View run First dataset at: http://127.0.0.1:8080/#/experiments/0/runs/f1f4e9f64a7e46eab3e2d6c8742e14a6
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


## Второй эксперимент (Простая очистка от знаков препинания и изменение эмодзи)

In [11]:
def clean_text_only(text):

    if not isinstance(text, str):
        return ""
    
    text = text.lower()
    
    # Заменяем эмодзи
    text = emoji.demojize(text, delimiters=(" ", " "))
    
    # Удаляем переносы и спецсимволы
    text = re.sub(r'[\n\r\t]', ' ', text)
    
    # Удаляем пунктуацию
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Убираем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [12]:
with mlflow.start_run(run_name = 'Second dataset'):
    
    mlflow.set_tag('Dataset_version', '2.0.0')

    df_annotations_1 = pd.read_json("project-8-at-2025-11-01-11-43-483e368c.json")
    df_annotations_2 = pd.read_json("project-9-at-2025-11-01-11-51-0428f45f.json")
    df_annotations_3 = pd.read_json("project-10-at-2025-11-06-14-16-bb9bc9cd.json")

    df_annotations = pd.concat([df_annotations_1, df_annotations_2, df_annotations_3])

    df = pd.DataFrame(columns = ['span', 'label'])

    for mark in df_annotations['aspect_sentiment']:
        for param in range(len(mark)):
            span = mark[param]['text']
            label = mark[param]['labels'][0]

            df.loc[len(df)] = [span, label]
    
    df['span'] = df['span'].apply(clean_text_only)

    file = df.to_csv('Second_version.csv')
    
    display(df)

    mlflow.log_artifact('Second_version.csv', 'datasets')


    with open('preprocess_text_second.pkl', 'wb') as f:
        cloudpickle.dump(clean_text_only, f)

    mlflow.log_artifact('preprocess_text_second.pkl', 'functions')

,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,хотелось купить очень много но в последнее вре...,O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...
2063,check_mark_button итог nчипсы лейс flamin hot ...,ВКУС_POSITIVE
2064,у них привлекательный и аппетитный вид,ПАЧКА_POSITIVE
2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,к покупке рекомендую,ВКУС_POSITIVE


🏃 View run Second dataset at: http://127.0.0.1:8080/#/experiments/0/runs/fe1cee81afdc40208ab2bc72e140f051
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


## Третий эксперимент (Никакой очистки)

In [13]:
with mlflow.start_run(run_name = 'Third dataset'):
    
    mlflow.set_tag('Dataset_version', '3.0.0')

    df_annotations_1 = pd.read_json("project-8-at-2025-11-01-11-43-483e368c.json")
    df_annotations_2 = pd.read_json("project-9-at-2025-11-01-11-51-0428f45f.json")
    df_annotations_3 = pd.read_json("project-10-at-2025-11-06-14-16-bb9bc9cd.json")

    df_annotations = pd.concat([df_annotations_1, df_annotations_2, df_annotations_3])

    df = pd.DataFrame(columns = ['span', 'label'])

    for mark in df_annotations['aspect_sentiment']:
        for param in range(len(mark)):
            span = mark[param]['text']
            label = mark[param]['labels'][0]

            df.loc[len(df)] = [span, label]
    
    file = df.to_csv('Third_version.csv')
    
    display(df)

    mlflow.log_artifact('Third_version.csv', 'datasets')

,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,"Я ел эти чипсы очень долго, еще года 2 назад с...",O
2,"хотелось купить очень много, но в последнее вр...",O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал НАМНОГО острее обычного,ВКУС_NEGATIVE
...,...,...
2063,"✅ ИТОГ\nЧипсы Лейс Flamin Hot ""Острая креветка...",ВКУС_POSITIVE
2064,У них привлекательный и аппетитный вид,ПАЧКА_POSITIVE
2065,оригинальный пикантный вкус.,ВКУС_POSITIVE
2066,К покупке рекомендую!,ВКУС_POSITIVE


🏃 View run Third dataset at: http://127.0.0.1:8080/#/experiments/0/runs/85b7f73a97c046b1939da0fc3eb2a90c
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


# Эксперементирование с моделями

## Первый эксперимент (Простая модель)

Будем обучать простую модель: "Наивный Байесовский Классификатор" из `Sklearn`, будем проводить тест на 3 версиях датасета и поймем, какая модель лучше себя покажет при работе с тем или иным датасетом

### Первый датасет

In [14]:
with mlflow.start_run(run_name = 'first_model_experiment'):

    mlflow.set_tag('NaiveBayes', '1.0.0')

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=['0'],  # Default эксперимент всегда имеет ID = 0
        filter_string="tags.mlflow.runName = 'First dataset'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id

    files = client.list_artifacts(first_dataset_latest_run_id, 'datasets')

    dataframe = files[0].path

    dataframe_path = client.download_artifacts(first_dataset_latest_run_id, dataframe)

    df = pd.read_csv(dataframe_path)

    display(df)



    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectoraizer = TfidfVectorizer(lowercase=True,
                                  analyzer = 'word',
                                  max_features=5000,
                                  ngram_range=(1, 4),
                                  min_df=2,
                                  max_df=0.9
                                  )
    
    encoder = LabelEncoder()



    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST/VAL
    # =====================================================================================================================================


    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

    X_train = df_train['span']
    X_train = vectoraizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)


    X_test = df_test['span']
    X_test = vectoraizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)


    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog()

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print(f'f1-score равен {f1_score(y_test, y_pred, average='weighted')}')
    print(f'accuracy-score равен {accuracy_score(y_test, y_pred)}')

    joblib.dump(model, 'naive_bayes.pkl')
    joblib.dump(vectoraizer, 'tfidfvectoraizer.pkl')
    joblib.dump(encoder, 'labelencoder.pkl')
    
    mlflow.log_artifact('tfidfvectoraizer.pkl', 'models')
    mlflow.log_artifact('labelencoder.pkl', 'models')
    mlflow.log_artifact('naive_bayes.pkl', 'models')

,Unnamed: 0,span,label
0,0,вкус реально отличный,ВКУС_POSITIVE
1,1,я есть чипсы очень долго год 2 назад советоват...,O
2,2,хотеться купить очень последний время стать оч...,O
3,3,вкус иметь каждый 2 пачка,O
4,4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...,...
2063,2063,итогчипсы лейс flamin hot острый креветка васа...,ВКУС_POSITIVE
2064,2064,у привлекательный аппетитный вид,ПАЧКА_POSITIVE
2065,2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,2066,к покупка рекомендовать,ВКУС_POSITIVE


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


f1-score равен 0.32907258187514044
accuracy-score равен 0.4033816425120773
🏃 View run first_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/5b744dd079fa4e4ebdbc52836203b4ce
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [15]:
client = MlflowClient()


latest_run_model = client.search_runs(experiment_ids=['0'],
                         filter_string='attribute.run_name = "first_model_experiment"',
                         order_by=['attribute.end_time desc'])

latest_run_model_id = latest_run_model[0].info.run_id

print(latest_run_model_id)

model_run = client.get_run(latest_run_model_id)

artifacts = client.list_artifacts(latest_run_model_id, 'models')

# Получаем путь к нашим моделям из MLFlow

vectoraizer_file = client.download_artifacts(latest_run_model_id, 'models/tfidfvectoraizer.pkl')
bayes_file = client.download_artifacts(latest_run_model_id, 'models/naive_bayes.pkl')
encoder_file = client.download_artifacts(latest_run_model_id, 'models/labelencoder.pkl')


# Получаем путь к нашей функции предобработки данных из MLFlow


latest_run_dataset = client.search_runs(experiment_ids=['0'],
                                        filter_string='attributes.run_name="First dataset"',
                                        order_by=['attributes.end_time desc'])


latest_run_dataset_id = latest_run_dataset[0].info.run_id

art_loc = client.download_artifacts(latest_run_dataset_id, 'functions/preprocess_text_first.pkl')


# Ипортируем их в виде выполняемого скрипта

vectoraizer = joblib.load(vectoraizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)


# Предсказываем метку текста

text = 'Меня привлекло такое сочетание'

preprocessed_text = preprocess_func(text)

done_text = vectoraizer.transform([preprocessed_text])
    
label = bayes.predict(done_text)

print('=' * 100)

print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(encoder.inverse_transform(label))



5b744dd079fa4e4ebdbc52836203b4ce


Исходный текст: "Меня привлекло такое сочетание"
Предобработанный текст: "я привлечь такой сочетание"
['ВКУС_POSITIVE']


### Второй датасет

In [16]:
with mlflow.start_run(run_name = 'second_model_experiment'):

    mlflow.set_tag('NaiveBayes', '2.0.0')

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=['0'],  # Default эксперимент всегда имеет ID = 0
        filter_string="tags.mlflow.runName = 'Second dataset'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id

    files = client.list_artifacts(first_dataset_latest_run_id, 'datasets')

    dataframe = files[0].path

    dataframe_path = client.download_artifacts(first_dataset_latest_run_id, dataframe)

    df = pd.read_csv(dataframe_path)

    display(df)



    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectoraizer = TfidfVectorizer(lowercase=True,
                                  analyzer = 'word',
                                  max_features=5000,
                                  ngram_range=(1, 4),
                                  min_df=2,
                                  max_df=0.9
                                  )
    
    encoder = LabelEncoder()



    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST/VAL
    # =====================================================================================================================================


    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

    X_train = df_train['span']
    X_train = vectoraizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)


    X_test = df_test['span']
    X_test = vectoraizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)


    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog()

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print(f'f1-score равен {f1_score(y_test, y_pred, average='weighted')}')
    print(f'accuracy-score равен {accuracy_score(y_test, y_pred)}')

    joblib.dump(model, 'naive_bayes.pkl')
    joblib.dump(vectoraizer, 'tfidfvectoraizer.pkl')
    joblib.dump(encoder, 'labelencoder.pkl')
    
    mlflow.log_artifact('tfidfvectoraizer.pkl', 'models')
    mlflow.log_artifact('labelencoder.pkl', 'models')
    mlflow.log_artifact('naive_bayes.pkl', 'models')

,Unnamed: 0,span,label
0,0,вкус был реально отличный,ВКУС_POSITIVE
1,1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,2,хотелось купить очень много но в последнее вре...,O
3,3,сейчас такой вкус имеет каждая 2 пачка,O
4,4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...,...
2063,2063,check_mark_button итог nчипсы лейс flamin hot ...,ВКУС_POSITIVE
2064,2064,у них привлекательный и аппетитный вид,ПАЧКА_POSITIVE
2065,2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,2066,к покупке рекомендую,ВКУС_POSITIVE


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


f1-score равен 0.31295994125933585
accuracy-score равен 0.37922705314009664
🏃 View run second_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/448ebe798ba94371abc1f9c5491c9de0
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [17]:
client = MlflowClient()


latest_run_model = client.search_runs(experiment_ids=['0'],
                         filter_string='attribute.run_name = "second_model_experiment"',
                         order_by=['attribute.end_time desc'])

latest_run_model_id = latest_run_model[0].info.run_id

print(latest_run_model_id)

model_run = client.get_run(latest_run_model_id)

artifacts = client.list_artifacts(latest_run_model_id, 'models')

# Получаем путь к нашим моделям из MLFlow

vectoraizer_file = client.download_artifacts(latest_run_model_id, 'models/tfidfvectoraizer.pkl')
bayes_file = client.download_artifacts(latest_run_model_id, 'models/naive_bayes.pkl')
encoder_file = client.download_artifacts(latest_run_model_id, 'models/labelencoder.pkl')


# Получаем путь к нашей функции предобработки данных из MLFlow


latest_run_dataset = client.search_runs(experiment_ids=['0'],
                                        filter_string='attributes.run_name="Second dataset"',
                                        order_by=['attributes.end_time desc'])


latest_run_dataset_id = latest_run_dataset[0].info.run_id

art_loc = client.download_artifacts(latest_run_dataset_id, 'functions/preprocess_text_second.pkl')


# Ипортируем их в виде выполняемого скрипта

vectoraizer = joblib.load(vectoraizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)


# Предсказываем метку текста

text = 'Меня не привлекло такое сочетание 😭'

preprocessed_text = preprocess_func(text)

done_text = vectoraizer.transform([preprocessed_text])
    
label = bayes.predict(done_text)

print('=' * 100)

print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(encoder.inverse_transform(label))

448ebe798ba94371abc1f9c5491c9de0


Исходный текст: "Меня не привлекло такое сочетание 😭"
Предобработанный текст: "меня не привлекло такое сочетание loudly_crying_face"
['ВКУС_POSITIVE']


### Третий датасет

In [18]:
with mlflow.start_run(run_name = 'third_model_experiment'):

    mlflow.set_tag('NaiveBayes', '3.0.0')

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=['0'],  # Default эксперимент всегда имеет ID = 0
        filter_string="tags.mlflow.runName = 'Third dataset'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id

    files = client.list_artifacts(first_dataset_latest_run_id, 'datasets')

    dataframe = files[0].path

    dataframe_path = client.download_artifacts(first_dataset_latest_run_id, dataframe)

    df = pd.read_csv(dataframe_path)

    display(df)



    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectoraizer = TfidfVectorizer(lowercase=True,
                                  analyzer = 'word',
                                  max_features=5000,
                                  ngram_range=(1, 4),
                                  min_df=2,
                                  max_df=0.9
                                  )
    
    encoder = LabelEncoder()



    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST/VAL
    # =====================================================================================================================================


    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

    X_train = df_train['span']
    X_train = vectoraizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)


    X_test = df_test['span']
    X_test = vectoraizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)


    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog()

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print(f'f1-score равен {f1_score(y_test, y_pred, average='weighted')}')
    print(f'accuracy-score равен {accuracy_score(y_test, y_pred)}')

    joblib.dump(model, 'naive_bayes.pkl')
    joblib.dump(vectoraizer, 'tfidfvectoraizer.pkl')
    joblib.dump(encoder, 'labelencoder.pkl')
    
    mlflow.log_artifact('tfidfvectoraizer.pkl', 'models')
    mlflow.log_artifact('labelencoder.pkl', 'models')
    mlflow.log_artifact('naive_bayes.pkl', 'models')

,Unnamed: 0,span,label
0,0,вкус был реально отличный,ВКУС_POSITIVE
1,1,"Я ел эти чипсы очень долго, еще года 2 назад с...",O
2,2,"хотелось купить очень много, но в последнее вр...",O
3,3,сейчас такой вкус имеет каждая 2 пачка,O
4,4,вкус которых стал НАМНОГО острее обычного,ВКУС_NEGATIVE
...,...,...,...
2063,2063,"✅ ИТОГ\nЧипсы Лейс Flamin Hot ""Острая креветка...",ВКУС_POSITIVE
2064,2064,У них привлекательный и аппетитный вид,ПАЧКА_POSITIVE
2065,2065,оригинальный пикантный вкус.,ВКУС_POSITIVE
2066,2066,К покупке рекомендую!,ВКУС_POSITIVE


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


f1-score равен 0.31762069535565496
accuracy-score равен 0.38164251207729466
🏃 View run third_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/b8c2c44296e747ff82fc1d01712e92b3
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатоы

In [19]:
client = MlflowClient()


latest_run_model = client.search_runs(experiment_ids=['0'],
                         filter_string='attribute.run_name = "third_model_experiment"',
                         order_by=['attribute.end_time desc'])

latest_run_model_id = latest_run_model[0].info.run_id

print(latest_run_model_id)

model_run = client.get_run(latest_run_model_id)

artifacts = client.list_artifacts(latest_run_model_id, 'models')

# Получаем путь к нашим моделям из MLFlow

vectoraizer_file = client.download_artifacts(latest_run_model_id, 'models/tfidfvectoraizer.pkl')
bayes_file = client.download_artifacts(latest_run_model_id, 'models/naive_bayes.pkl')
encoder_file = client.download_artifacts(latest_run_model_id, 'models/labelencoder.pkl')


# Ипортируем их в виде выполняемого скрипта

vectoraizer = joblib.load(vectoraizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)


# Предсказываем метку текста

text = 'Меня не привлекло такое сочетание 😭'

preprocessed_text = preprocess_func(text)

done_text = vectoraizer.transform([preprocessed_text])
    
label = bayes.predict(done_text)

print('=' * 100)

print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(encoder.inverse_transform(label))

b8c2c44296e747ff82fc1d01712e92b3


Исходный текст: "Меня не привлекло такое сочетание 😭"
Предобработанный текст: "меня не привлекло такое сочетание loudly_crying_face"
['ВКУС_POSITIVE']


### Итог

In [20]:
results = pd.DataFrame({
    'Версия': ['Версия 1', 'Версия 2', 'Версия 3'],
    'F1-Score': [0.363707, 0.344306, 0.344139],
    'Accuracy': [0.413043, 0.391304, 0.391304],
    'Описание': ['Лемматизация, удаление знаков пунктуации', 'Очистка от знаков препинания и emoji to text', 'Никакой очистки']
})

results

,Версия,F1-Score,Accuracy,Описание
0,Версия 1,0.363707,0.413043,"Лемматизация, удаление знаков пунктуации"
1,Версия 2,0.344306,0.391304,Очистка от знаков препинания и emoji to text
2,Версия 3,0.344139,0.391304,Никакой очистки


## Второй эксперимент (LSTM)

### Первый датасет

In [21]:
with mlflow.start_run(run_name='first_experiment_neural_network'):
    
    mlflow.set_tag('LSTM', '1.0.0')

    
    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================


    client = MlflowClient()

    first_dataset_run = client.search_runs(experiment_ids=['0'],
                                           filter_string = "attributes.run_name='First dataset'",
                                           order_by = ['attributes.end_time desc'])
    
    first_dataset_run_latest = first_dataset_run[0]
    
    first_dataset_run_id = first_dataset_run_latest.info.run_id
    art = client.download_artifacts(first_dataset_run_id, 'datasets/First_version.csv')

    df = pd.read_csv(art)

    display(df)


    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    mlflow.tensorflow.autolog()

    tokenizer = Tokenizer(num_words = 2000, 
                          oov_token = '<OOV>',
                          filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                          lower=True,
                          split=' ',
                          char_level=False)

    tokenizer.fit_on_texts(df['span'])


    df_train, df_temp = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

    df_test, df_val = train_test_split(df_temp, test_size=0.5, random_state=42, stratify=df_temp['label'])
    

    X_train_vec = tokenizer.texts_to_sequences(df_train['span'])
    X_test_vec = tokenizer.texts_to_sequences(df_test['span'])
    X_val_vec = tokenizer.texts_to_sequences(df_val['span'])


    max_len_text = 0

    for i in df['span']:
        max_len_text = max(max_len_text, len(i.split(' ')))

    print(max_len_text)


    X_train_pad = pad_sequences(X_train_vec, max_len_text, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_vec, max_len_text, padding='post', truncating='post')
    X_val_pad = pad_sequences(X_val_vec, max_len_text, padding='post', truncating='post')


    # =====================================================================================================================================
    #                                              ПОДГОТОВКА ТАРГЕТОВ
    # =====================================================================================================================================


    encoder = LabelEncoder()

    y_train = encoder.fit_transform(df_train['label'])
    y_test = encoder.transform(df_test['label'])
    y_val = encoder.transform(df_val['label'])



    # =====================================================================================================================================
    #                                              СОЗДАНИЕ МОДЕЛИ
    # =====================================================================================================================================


    model = Sequential()

    model.add(Embedding(input_dim = 2000, output_dim = 128, input_length = max_len_text))

    model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.3)))

    model.add(Dropout(0.5))

    model.add(Dense(10, activation = 'softmax'))

    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

    model.fit(X_train_pad, y_train, batch_size = 16, epochs = 10, validation_data=(X_val_pad, y_val))

    model.evaluate(X_test_pad, y_test)

    model.summary()

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    pred_proba = model.predict(X_test_pad)

    pred_class = np.argmax(pred_proba, axis = 1)
    
    print(f'f1-score у модели равен {f1_score(pred_class, y_test, average = 'weighted')}')

    mlflow.log_metric('f1', f1_score(pred_class, y_test, average = 'weighted'))

    model.save('LSTM_ver_1.keras')

    with open('tokenizer.pkl', 'wb') as f:
        cloudpickle.dump(tokenizer, f)

    
    with open('encoder.pkl', 'wb') as f:
        cloudpickle.dump(encoder, f)


    mlflow.log_artifact('LSTM_ver_1.keras', 'models')
    mlflow.log_artifact('tokenizer.pkl', 'models') 
    mlflow.log_artifact('encoder.pkl', 'models')


,Unnamed: 0,span,label
0,0,вкус реально отличный,ВКУС_POSITIVE
1,1,я есть чипсы очень долго год 2 назад советоват...,O
2,2,хотеться купить очень последний время стать оч...,O
3,3,вкус иметь каждый 2 пачка,O
4,4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...,...
2063,2063,итогчипсы лейс flamin hot острый креветка васа...,ВКУС_POSITIVE
2064,2064,у привлекательный аппетитный вид,ПАЧКА_POSITIVE
2065,2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,2066,к покупка рекомендовать,ВКУС_POSITIVE


73


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025/11/14 14:15:12 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: Sequential model 'sequential' has no defined input shape yet.


Epoch 1/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.2275 - loss: 2.1984

104/104 ━━━━━━━━━━━━━━━━━━━━ 53s 206ms/step - accuracy: 0.2255 - loss: 2.1665 - val_accuracy: 0.2319 - val_loss: 2.0960
Epoch 2/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.2287 - loss: 2.0806

...

 Total params: 1,068,320 (4.08 MB)

 Trainable params: 356,106 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 712,214 (2.72 MB)

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step
f1-score у модели равен 0.4556642959166068
🏃 View run first_experiment_neural_network at: http://127.0.0.1:8080/#/experiments/0/runs/bb16c224144d4b47a34f1365031d3cf4
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [22]:
client = MlflowClient()

last_model_runs = client.search_runs(experiment_ids=['0'],
                                    filter_string="attributes.run_name = 'first_experiment_neural_network'",
                                    order_by=['attributes.end_time desc'])


last_model_run_id = last_model_runs[0].info.run_id

nn_loc = client.download_artifacts(last_model_run_id, 'models/LSTM_ver_1.keras')
encoder_loc = client.download_artifacts(last_model_run_id, 'models/encoder.pkl')
tokenizer_loc = client.download_artifacts(last_model_run_id, 'models/tokenizer.pkl')

model_keras = keras.models.load_model(nn_loc)
encoder = joblib.load(encoder_loc)
tokenizer = joblib.load(tokenizer_loc)


# Инференс модели

text = 'В целом, чипсы Ашан Красная птица Барбекю вполне съедобные'

tokenized_text = tokenizer.texts_to_sequences([text])

max_len = model_keras.input_shape[1]
padded_text = pad_sequences(tokenized_text, maxlen=max_len, padding='post', truncating='post')

pred = model_keras.predict(padded_text)

predicted_class_ind = np.argmax(pred, axis=1)
predicted_class = encoder.inverse_transform(predicted_class_ind)

print(predicted_class)

c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 11 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 987ms/step
['O']


### Второй датасет

In [23]:
with mlflow.start_run(run_name='second_experiment_neural_network'):
    
    mlflow.set_tag('LSTM', '2.0.0')

    
    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================


    client = MlflowClient()

    first_dataset_run = client.search_runs(experiment_ids=['0'],
                                           filter_string = "attributes.run_name='Second dataset'",
                                           order_by = ['attributes.end_time desc'])
    
    first_dataset_run_latest = first_dataset_run[0]
    
    first_dataset_run_id = first_dataset_run_latest.info.run_id
    art = client.download_artifacts(first_dataset_run_id, 'datasets/Second_version.csv')

    df = pd.read_csv(art)

    display(df)

        
    # =====================================================================================================================================
    #                                         ТОКЕНИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================


    mlflow.tensorflow.autolog()

    tokenizer = Tokenizer(num_words = 2000, 
                          filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                          lower=True,
                          split=' ',
                          char_level=False,
                          oov_token=None,
                          analyzer=None)
    
    tokenizer.fit_on_texts(df['span'])

    df_train, df_temp = train_test_split(df, test_size = 0.2, random_state = 42)
    df_test, df_val = train_test_split(df_temp, test_size=0.5, random_state = 42)



    X_train_vec = tokenizer.texts_to_sequences(df_train['span'])
    X_test_vec = tokenizer.texts_to_sequences(df_test['span'])
    X_val_vec = tokenizer.texts_to_sequences(df_val['span'])

    X_train_pad = pad_sequences(X_train_vec, maxlen=100, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_vec, maxlen=100, padding='post', truncating='post')
    X_val_pad = pad_sequences(X_val_vec, maxlen=100, padding='post', truncating='post')
    


    # =====================================================================================================================================
    #                                         ПОДГОТОВКА ТАРГЕТОВ
    # =====================================================================================================================================


    encoder = LabelEncoder()

    y_train = encoder.fit_transform(df_train['label'])
    y_test = encoder.transform(df_test['label'])
    y_val = encoder.transform(df_val['label'])


    # =====================================================================================================================================
    #                                         СОЗДАНИЕ МОДЕЛИ
    # =====================================================================================================================================


    model = Sequential()

    model.add(Embedding(input_dim = 2000, output_dim = 64, input_length = 100))

    model.add(Conv1D(32, 3, activation='relu'))

    model.add(Dropout(0.5))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(16, activation = 'relu'))

    model.add(Dropout(0.2))

    model.add(Dense(10, activation = 'softmax'))

    model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics = ['accuracy'])

    model.fit(X_train_pad, y_train, batch_size=16, epochs=40, validation_data = (X_val_pad, y_val))

    model.summary()

    model.evaluate(X_test_pad, y_test)

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    pred = model.predict(X_test_pad)

    pred_class = np.argmax(pred, axis=1)

    print(f'f1-score у модели равен {f1_score(pred_class, y_test, average = 'weighted')}')

    mlflow.log_metric('f1', f1_score(pred_class, y_test, average = 'weighted'))

    model.save('LSTM_ver_2.keras')

    joblib.dump(tokenizer, 'tokenizer.pkl')
    joblib.dump(encoder, 'encoder.pkl')

    mlflow.log_artifact('tokenizer.pkl', 'models')
    mlflow.log_artifact('encoder.pkl', 'models')
    mlflow.log_artifact('LSTM_ver_2.keras', 'models')    



,Unnamed: 0,span,label
0,0,вкус был реально отличный,ВКУС_POSITIVE
1,1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,2,хотелось купить очень много но в последнее вре...,O
3,3,сейчас такой вкус имеет каждая 2 пачка,O
4,4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...,...
2063,2063,check_mark_button итог nчипсы лейс flamin hot ...,ВКУС_POSITIVE
2064,2064,у них привлекательный и аппетитный вид,ПАЧКА_POSITIVE
2065,2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,2066,к покупке рекомендую,ВКУС_POSITIVE


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025/11/14 14:19:47 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: Sequential model 'sequential_1' has no defined input shape yet.


Epoch 1/40
100/104 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1403 - loss: 2.2957

104/104 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.1808 - loss: 2.2741 - val_accuracy: 0.2657 - val_loss: 2.2133
Epoch 2/40
 99/104 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2390 - loss: 2.1599

104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2527 - loss: 2.1250 - val_accuracy: 0.2995 - val_loss: 2.0611
Epoch 3/40
101/104 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3188 - loss: 1.9072

...

 Trainable params: 134,874 (526.85 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 269,750 (1.03 MB)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3575 - loss: 4.1014 


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step  
f1-score у модели равен 0.352281074230904
🏃 View run second_experiment_neural_network at: http://127.0.0.1:8080/#/experiments/0/runs/96e203d967044ad4ac9f0267b29b3ccb
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


## Третий эксперимент (Трансформеры)

### Первый датасет

In [ ]:
with mlflow.start_run(run_name='transformers_experiment_1'):
    client = MlflowClient()

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================
    
    
    first_dataset_runs = client.search_runs(experiment_ids=['0'],
                             filter_string="attributes.run_name = 'First dataset'",
                             order_by=['attributes.end_time desc'])
    
    need_run = first_dataset_runs[0]
    need_run_id = need_run.info.run_id
    art_loc = client.download_artifacts(need_run_id, 'datasets/First_version.csv')
    df = pd.read_csv(art_loc)
    df = df[['span', 'label']]
    df = df.rename(columns={"label": "labels"})

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА ДАТАСЕТА/ТАРГЕТОВ
    # =====================================================================================================================================
    
    
    df_train, df_temp = train_test_split(df, test_size=0.2, random_state=42, stratify=df['labels'])
    df_test, df_val = train_test_split(df_temp, test_size=0.5, random_state=42, stratify=df_temp['labels'])

    encoder = LabelEncoder()
    y_train = encoder.fit_transform(df_train['labels'])
    y_test = encoder.transform(df_test['labels'])
    y_val = encoder.transform(df_val['labels'])


    # =====================================================================================================================================
    #                                         ПОДГОТОВКА МОДЕЛИ
    # =====================================================================================================================================
    

    mlflow.transformers.autolog()

    model_name = 'cointegrated/rubert-tiny2'
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, 
        num_labels=len(encoder.classes_), 
        id2label={i: label for i, label in enumerate(encoder.classes_)},
        label2id={label: i for i, label in enumerate(encoder.classes_)}
    )
    
    dataset_train = Dataset.from_pandas(df_train.assign(labels=y_train))    
    dataset_test = Dataset.from_pandas(df_test.assign(labels=y_test))    
    dataset_val = Dataset.from_pandas(df_val.assign(labels=y_val))

    def tokenize_dataset(row):
        return tokenizer(
            row['span'],
            truncation=True, 
            padding=False,
            max_length=512,
            return_tensors=None
        )

    dataset_tokenized_train = dataset_train.map(tokenize_dataset, batched=False)
    dataset_tokenized_test = dataset_test.map(tokenize_dataset, batched=False)
    dataset_tokenized_val = dataset_val.map(tokenize_dataset, batched=False)


    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding=True,
        return_tensors='pt'
    )
    
    training_args = TrainingArguments(
        output_dir='./models/',
        overwrite_output_dir=True,
        logging_dir='./logs/',
        num_train_epochs=1,
        learning_rate=3e-5,
        per_device_train_batch_size=4,   
        eval_strategy='epoch',
        save_strategy='epoch',
        warmup_ratio=0.1,
        lr_scheduler_type='cosine',
        metric_for_best_model='f1-score',
        weight_decay=0.2,
        load_best_model_at_end=True,
        save_total_limit=2,
        max_grad_norm=1.0,
        logging_steps=100
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        
        return {
            'f1-score': f1_score(labels, predictions, average='weighted'),
            'accuracy': accuracy_score(labels, predictions)
        }

    trainer = Trainer(
        model=model, 
        args=training_args, 
        train_dataset=dataset_tokenized_train,
        eval_dataset=dataset_tokenized_val,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

    trainer.train()

    final_metrics = trainer.evaluate(dataset_tokenized_test)
    
    model_dir = "./Transformers_ver_1"
    tokenizer_dir = "./Tokenizer_transformers_ver_1"
    
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(tokenizer_dir, exist_ok=True)
    
    model.save_pretrained(model_dir)
    tokenizer.save_pretrained(tokenizer_dir)

    mlflow.log_artifacts(model_dir, "model")
    mlflow.log_artifacts(tokenizer_dir, "tokenizer")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 207/207 [00:00<00:00, 2401.11 examples/s]


Train features: ['span', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask']
Val features: ['span', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask']


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,F1-score,Accuracy
1,1.960500,1.936422,0.230187,0.338164


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


🏃 View run transformers_experiment_1 at: http://127.0.0.1:8080/#/experiments/0/runs/15a40ee889404e4180950782979c694b
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0
